In [267]:
import pybullet as p
import pybullet_data
import numpy as np
import kinpy as kp

In [268]:
model = kp.build_serial_chain_from_urdf(open("arm.urdf"),'arm_link_5')
jacobians = model.jacobian([0,0,0,0,0,0],end_only=False)

In [269]:
print(jacobians)

{'base_link': array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]]), 'arm_link_0': array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]]), 'arm_link_1': array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.]]), 'arm_link_2': array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.]]), 'arm_link_3': array([[0.   , 0.317, 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   ,

In [270]:
print(model.get_joint_parameter_names())

['arm_joint_1', 'arm_joint_2', 'arm_joint_3', 'arm_joint_4']


In [271]:
print(model)

base_link_frame
└──── arm_link_0_frame
      └──── arm_link_1_frame
            └──── arm_link_2_frame
                  └──── arm_link_3_frame
                        └──── arm_link_4_frame
                              └──── arm_link_5_frame


In [272]:
print(model.find_link("arm_link_5"))

Link(name='arm_link_5', offset=Transform(rot=[1. 0. 0. 0.], pos=[0. 0. 0.]), visuals=[Visual(offset=Transform(rot=[1. 0. 0. 0.], pos=[0. 0. 0.]), geom_type='None', geom_param=None)])


In [273]:
def setJointPosition(robot, position, kp=1.0, kv=0.3):
  num_joints = p.getNumJoints(robot)
  zero_vec = [0.0] * num_joints
  if len(position) == num_joints:
    p.setJointMotorControlArray(robot,
                                range(num_joints),
                                p.POSITION_CONTROL,
                                targetPositions=position,
                                targetVelocities=zero_vec,
                                positionGains=[kp] * num_joints,
                                velocityGains=[kv] * num_joints)
  else:
    print("Not setting torque. "
          "Expected torque vector of "
          "length {}, got {}".format(num_joints, len(torque)))
def getJointStates(robot):
  joint_states = p.getJointStates(robot, range(p.getNumJoints(robot)))
  joint_positions = [state[0] for state in joint_states]
  joint_velocities = [state[1] for state in joint_states]
  joint_torques = [state[3] for state in joint_states]
  return joint_positions, joint_velocities, joint_torques

def getMotorJointStates(robot):
  joint_states = p.getJointStates(robot, range(p.getNumJoints(robot)))
  joint_infos = [p.getJointInfo(robot, i) for i in range(p.getNumJoints(robot))]
  joint_states = [j for j, i in zip(joint_states, joint_infos) if i[3] > -1]
  joint_positions = [state[0] for state in joint_states]
  joint_velocities = [state[1] for state in joint_states]
  joint_torques = [state[3] for state in joint_states]
  return joint_positions, joint_velocities, joint_torques



In [274]:
clid = p.connect(p.SHARED_MEMORY)
if (clid < 0):
  p.connect(p.DIRECT)
  
p.setAdditionalSearchPath(pybullet_data.getDataPath())

time_step = 0.001
gravity_constant = -9.81
p.resetSimulation()
p.setTimeStep(time_step)
p.setGravity(0.0, 0.0, gravity_constant)

p.loadURDF("plane.urdf", [0, 0, -0.3])

# kukaId = p.loadURDF("TwoJointRobot_w_fixedJoints.urdf", useFixedBase=True)
kukaId = p.loadURDF("arm.urdf", useFixedBase=True)
p.resetBasePositionAndOrientation(kukaId, [0, 0, 0], [0, 0, 0, 1])
numJoints = p.getNumJoints(kukaId)

print("Number of joints: ", numJoints)
kukaEndEffectorIndex = numJoints - 1

# Set a joint target for the position control and step the sim.
setJointPosition(kukaId, [0.0] * numJoints)
p.stepSimulation()

# Get the joint and link state directly from Bullet.
pos, vel, torq = getJointStates(kukaId)
mpos, mvel, mtorq = getMotorJointStates(kukaId)

result = p.getLinkState(kukaId,
                        kukaEndEffectorIndex,
                        computeLinkVelocity=1,
                        computeForwardKinematics=1)


link_trn, link_rot, com_trn, com_rot, frame_pos, frame_rot, link_vt, link_vr = result

print(result)

Number of joints:  6
((0.0, 0.0, 0.857), (0.0, 0.0, 0.0, 1.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0, 1.0), (0.0, 0.0, 0.8569999933242798), (0.0, 0.0, 0.0, 1.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0))


In [275]:
jointPositions = [p.getJointState(kukaId, jointIndex)[0] for jointIndex in range(numJoints)]
jointVelocities = [p.getJointState(kukaId, jointIndex)[1] for jointIndex in range(numJoints)]
jointAccelerations = [p.getJointState(kukaId, jointIndex)[2] for jointIndex in range(numJoints)]

zero_vec = [0.0] * len(mpos)

print(mpos)
print(jointPositions[1:5])
print(jointVelocities[1:5])
print(jointAccelerations[1:5])

# Calculate the Jacobian matrix
Jacobian, _ = p.calculateJacobian(kukaId, kukaEndEffectorIndex, [0, 0, 0], jointPositions[1:5], jointVelocities[1:5], zero_vec)

# Print the Jacobian matrix
print("Jacobian Matrix:")
print(np.concatenate([Jacobian,_]))
print(np.array(_))
print("-------------")
print(jacobians['arm_link_5'])

[0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0]
[(0.0, 0.0, 0.0, 0.0, 0.0, 0.0), (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), (0.0, 0.0, 0.0, 0.0, 0.0, 0.0)]
Jacobian Matrix:
[[0.     0.6795 0.3625 0.1605]
 [0.     0.     0.     0.    ]
 [0.     0.     0.     0.    ]
 [0.     0.     0.     0.    ]
 [0.     1.     1.     1.    ]
 [1.     0.     0.     0.    ]]
[[0. 0. 0. 0.]
 [0. 1. 1. 1.]
 [1. 0. 0. 0.]]
-------------
[[0.     0.6795 0.3625 0.1605 0.     0.    ]
 [0.     0.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.     0.    ]
 [0.     1.     1.     1.     0.     0.    ]
 [1.     0.     0.     0.     0.     0.    ]]


In [276]:
i = 0
mass 
for i in range(7):
    mass.append( p.getDynamicsInfo(kukaId,i-1)[0])
total_mass = sum(mass)
keys = jacobians.keys()
print(keys)

dict_keys(['base_link', 'arm_link_0', 'arm_link_1', 'arm_link_2', 'arm_link_3', 'arm_link_4', 'arm_link_5'])


In [277]:
com_jac = np.zeros(jacobians['base_link'].shape)
counter = 0
for i in keys:
    com_jac += jacobians[i] * mass[counter]/total_mass
    counter += 1
print(com_jac)

[[0.         0.00974175 0.00379657 0.00168096 0.         0.        ]
 [0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.        ]
 [0.         0.03023926 0.01875452 0.0104733  0.         0.        ]
 [0.04558386 0.         0.         0.         0.         0.        ]]


In [278]:
com_jac_inv = np.linalg.pinv(com_jac)
print(com_jac_inv.T)

[[ 0.00000000e+00  2.47922553e+02 -2.69474011e+02 -2.33273072e+02
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00 -8.52564099e-13  9.65046557e-13  8.31120239e-13
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00 -4.71075005e+01  8.87232682e+01  7.26164230e+01
   0.00000000e+00  0.00000000e+00]
 [ 2.19375897e+01  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]


In [281]:
V_task = np.array([0,0,-0.43,1,1,1])  

joint_velocities = np.dot(com_jac_inv, V_task)
print(joint_velocities)

[ 21.93758969 -47.10750046  88.72326815  72.61642303   0.
   0.        ]


damped least squares
legs are massless